In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from unstructured.partition.auto import partition
from langchain.schema import Document
from IPython.display import display, HTML
import os
import pandas as pd
#from langchain.document_loaders import DirectoryLoader
#from langchain_community.document_loaders import TextLoader

# 1. Document Loading and Page Tracking
docs = []
doc_folder = r'C:\Users\admin\Documents\LLM\B1-B data'
for filename in os.listdir(doc_folder):
    filepath = os.path.join(doc_folder, filename)
    if os.path.isfile(filepath):
        elements = partition(filename=filepath)
        for i, element in enumerate(elements):
            # Extract text content and page information
            text = str(element) 
            page_number = element.metadata.page_number if element.metadata.page_number else 'N/A'  # Extract page info
            docs.append({"source": filename, "content": text, "page": page_number})

# 2. Chunking while Preserving Page Information
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4096, chunk_overlap=300)
all_splits = []
current_chunk = ""
current_metadata = {} 

for doc in docs:
    splits = text_splitter.split_text(doc['content'])
    for split in splits:
        if len(current_chunk) + len(split) <= 4096: 
            current_chunk += split + " " # Add to the current chunk
            current_metadata = {"source": doc['source'], "page": doc['page']} 
        else:
            all_splits.append(Document(page_content=current_chunk, metadata=current_metadata))
            current_chunk = split + " "
            current_metadata = {"source": doc['source'], "page": doc['page']}

# Append the last chunk
if current_chunk:
    all_splits.append(Document(page_content=current_chunk, metadata=current_metadata)) 

# 3. Vectorstore and Retriever Setup
model = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=model)
llm = ChatOllama(model="llama3.1:8b")  # Or your preferred LLM

In [2]:
# 4. RAG Function (Incorporating Contextual Compression)
def RAG(user_prompt, llm, vectorstore, stream=False, source_summaries=False, retrieval = 'contextual', top_k_hits = 5):
    # retrieval methods: contextual, cosine_similarity, both
    def format_docs(docs):
        return "\n\n".join(
            f"Source: {doc.metadata['source']} - Page: {doc.metadata.get('page', 'N/A')}\n\n{doc.page_content}" 
            for doc in docs
        )

    RAG_TEMPLATE = """
        This is a chat between a user and an artificial intelligence assistant. 
        The assistant gives helpful, detailed, and polite answers to the user's questions based on the context. 
        With regard to the source, 'embedded' means the content was embedded in the PDF file, and 'predicted' means the content was generated by an OCR model. 
        As such, there may be inaccuracies (spelling, spacing, missing symbols, etc.) in the predicted content, the embedded content, and the actual content. 
        The assistant should also indicate when the answer cannot be found in the context.

        When providing an answer, cite the source document and page number in parentheses where you found the relevant information, like this: ([Source: document_name, Page 1]). 
        If multiple sources contain relevant information, cite them all.

        <context>
        {context}
        </context>

        Answer the following question:

        {question}"""
    question = user_prompt
    rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)
    retriever = vectorstore.as_retriever()
    if retrieval == 'contextual' or retrieval == 'both':
        compressor = LLMChainExtractor.from_llm(llm) 
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor, base_retriever=retriever
        )
        qa_chain = (
            {"context": compression_retriever | format_docs, "question": RunnablePassthrough()} 
            | rag_prompt
            | llm
            | StrOutputParser() 
        )
        
        docs = compression_retriever.invoke(question)  # Invoke on the question 
    if retrieval == 'cosine_similarity' or retrieval == 'both':
        if retrieval != 'both':
            qa_chain = (
                {"context": retriever | format_docs, "question": RunnablePassthrough()}
                | rag_prompt
                | llm
                | StrOutputParser()
            )
        if retrieval == 'both':
            docs.extend(vectorstore.similarity_search(question, k = top_k_hits))
        else:
            docs = vectorstore.similarity_search(question, k = top_k_hits)

    if not docs:
        return "No relevant documents found", pd.DataFrame()

    source_data = []
    for doc in docs:
        source_data.append({
            "source": doc.metadata['source'], 
            "page": doc.metadata.get('page', 'N/A'),
            "content": doc.page_content 
        })

    if source_summaries:
        summaries = [llm.invoke(f'Summarize this in one or two sentences. Only state main point, nothing else. <{doc.page_content}> ').content for doc in docs]
        source_df = pd.DataFrame(source_data)
        source_df["short summary"] = summaries
    else:
        source_df = pd.DataFrame(source_data)

    if stream:
        for chunk in qa_chain.stream(question):
            print(chunk, end="", flush=True)
        return '', source_df
    else:
        result = qa_chain.invoke(question)
        return result, source_df


In [15]:
#TODO: make the content column a "link" that takes you to that spot in the paper
#TODO: Make a gradio gui

user_prompt = "Summarize the findings of the human factors study on Helmet Mounted Displays (HMD) for B-1B pilots."
result, sources_df = RAG(user_prompt, llm, vectorstore, stream=True, source_summaries=False, retrieval = 'both', top_k_hits = 3) 
print(result)
print('\nSource information:')
root_dir = doc_folder
doc_names = sources_df['source'].values
sources_df['source'] = sources_df['source'].apply(lambda x: root_dir + '\\' + x)


# Function to convert file paths to HTML links
def make_clickable(val,doc_name):
    return f'<a href="{val}" target="_blank">{doc_name}</a>'

# Apply the function to the 'File Path' column
for i in range(len(sources_df)):
    sources_df.loc[i, 'source'] = make_clickable(sources_df.loc[i, 'source'],doc_names[i])

# Display the DataFrame as HTML
display(HTML(sources_df.to_html(escape=False)))

Based on the provided context, I can summarize the findings of the human factors study on Helmet Mounted Displays (HMD) for B-1B pilots as follows:

The study aimed to evaluate the effectiveness of a conceptual HMD compared to Head Down Displays (HDDs) in the JDAM/LAR environment. The results indicate that the HMD has some benefits, such as providing steering information and improving target situational awareness. However, there are also several limitations and issues with the current design.

Key findings include:

* The HMD provides a good view of the horizon and allows for easier navigation, but it can be difficult to see plus ID threats at certain angles (Predicted).
* The symbology on the HMD is not as intuitive as course lines + DMPIs from a God's eye view or straight down viewpoint (Embedded).
* The display size, symbol motion, and overall viewability of the HMD information are somewhat acceptable, but the upper symbols are too small, and navigation info needs to be included (Pr

,source,page,content
0,A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf,11,"1. **INTRODUCTION**\n> 1.1 Over the last several years, the B-1B System Program Office Engineering Department (ASC/YDE) in conjunction with the Crew Station Evaluation Facility (CSEF) of the Engineering Directorate of Aeronautical Systems Center (ASC/ENFC) has conducted a series of applied human factors engineering studies.\n> 1.3 New smart weapons and advanced avionics systems have spurred the need for B-1B upgrades.\n\n2. **STUDY OBJECTIVE**\n> 2.1 The objective of this study was to conduct a human factors evaluation of a conceptual Helmet Mounted Display (HMD) compared to Head Down Displays (HDDs) in the JDAM/LAR environment.\n\n3. **METHOD**\n> 3.0 METHOD\n>>>\n> 3.1 The CSEF’s B-1B ERS was used.\n> 3.2 All study participants flew from the pilot’s seat.\n> 3.3 A 4” X 4” color liquid crystal display (LCD) and an 8” X 6” color LCD, or HDDs, as well as a HMD were operational on the pilot’s side.\n\n4. **HMD HEAD TRACKER**\n> 3.1.1 The HMD had a head tracker of the magnetic field type.\n> 3.1.2 Symbology within the HMD was determined by the helmet, ground, and aircraft spatial relationships in pitch, roll, and yaw and moved accordingly.\n\n5. **STUDY FINDINGS**\n(This part is not explicitly mentioned in the given context, but since it's a study on HMD, it's likely that some findings are discussed in the report, which might be included in a complete version of this document.)"
1,A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf,23,"4.3 Noteworthy question “11. All things considered, rate the OVERALL Operational Utility of a HMD in the B-1B. Use the following scale: RATING: average = \n\n5.1 Recommend that a HMD for the B-1B not be pursued further as mechanized herein. HDDs should be pursued. A god’s eye view of the world is preferable to a panoramic overlay on the world. There is a fundamental mismatch in trying to adapt a HMD, whose forte is the vertical situation, to rectify B-1B deficiencies in horizontal situational awareness. This mismatch may have safety implications such as disorientation and incorrect information interpretation.\n\n4.2 Noteworthy question “22. All things considered, rate the OVERALL Operational Utility of a 4” X 4” HDD in the B-1B. Use the following scale: RATING: average ="
2,A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf,35,"Context:\n\n10. All things considered, rate the Target Situational Awareness given by the HMD in the B-1B. Use the following scale:\n7 Totally Acceptable\n6 Very Acceptable\n5 Somewhat Acceptable\n4 Borderline\n3 Somewhat Unacceptable\n2 Very Unacceptable\n1 Totally Unacceptable\nRatings: 1 5 3 1\nRATING: Average: 5.4\n\n11. All things considered, rate the OVERALL Operational Utility of a HMD in the B- 1B. Use the following scale:\n7 Totally Acceptable\n6 Very Acceptable\n5 Somewhat Acceptable\n4 Borderline\n3 Somewhat Unacceptable\n2 Very Unacceptable\n1 Totally Unacceptable\nRatings: 1 5 3 1\nRATING: Average: 5.4"
3,A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf,30,"Here is the extracted relevant part of the context:\n\n> The HMD provides a steering arrow directly to the DMPIs. This allowed me to put the LAR over the DMPIs without OSO steering. With the HMD I actually knew that the weapons would come out.\n>\nThe ability to simply look to the right or left and see the DMPIs was very useful. It will help pilots operationally because they can quickly see a baring and distance relationship between the LARs and DMPIs.\n>\nNavigation info to the next steer point was good.\n\nThere are no parts of the context that seem directly related to Helmet Mounted Displays (HMD) for B-1B pilots specifically, but these few sentences appear relevant to answering the question about HMD for B-1B pilots."
4,A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf,11,"JHMCS 

In [13]:
# test code
#make dataframe an HTML object that has clickable links
#TODO: make the source open the doucment at that page. 
import os
from IPython.display import display, HTML

df = sources_df.copy()
root_dir = doc_folder
doc_names = df['source'].values
df['source'] = df['source'].apply(lambda x: root_dir + x )


# Function to convert file paths to HTML links
def make_clickable(val,doc_name):
    return f'<a href="{val}" target="_blank">{doc_name}</a>'

# Apply the function to the 'File Path' column
for i in range(len(df)):
    df.loc[i, 'source'] = make_clickable(df.loc[i, 'source'],doc_names[i])

# Display the DataFrame as HTML
display(HTML(df.to_html(escape=False)))

,source,page,content
0,"A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf"" target=""_blank"">A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf"" target=""_blank"">A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf"" target=""_blank"">A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf",11,"**Introduction**\n\n1.1 Over the last several years, the B-1B System Program Office Engineering Department (ASC/YDE) in conjunction with the Crew Station Evaluation Facility (CSEF) of the Engineering Directorate of Aeronautical Systems Center (ASC/ENFC) has conducted a series of applied human factors engineering studies.\n\n**2.0 STUDY OBJECTIVE**\n\n2.1 The objective of this study was to conduct a human factors evaluation of a conceptual Helmet Mounted Display (HMD) compared to Head Down Displays (HDDs) in the JDAM/LAR environment.\n\n**3.0 METHOD**\n\n3.1 The CSEF’s B-1B ERS was used. All study participants flew from the pilot’s seat. A 4” X 4” color liquid crystal display (LCD) and an 8” X 6” color LCD, or HDDs, as well as a HMD were operational on the pilot’s side.\n\n**3.1.1 HMD Head Tracker**\n\nThe HMD had a head tracker of the magnetic field type. The tracker determined the helmet, ground, and aircraft spatial relationships in pitch, roll, and yaw and moved the symbology within the HMD field of view accordingly."
1,"A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf"" target=""_blank"">A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf"" target=""_blank"">A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf"" target=""_blank"">A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf",23,"4.2 Noteworthy question– “1. All things considered, please RANK ORDER the following from an operational viewpoint (1 = best, 4 = worst): 1.4 - 4” X 4” HDD 1.8 - 8” X 6” HDD 2.8 - HMD 4.0 - Block D Displays”\n\n4.3 Noteworthy question “11. All things considered, rate the OVERALL Operational Utility of a HMD in the B-1B. Use the following scale: RATING: average = 5\n\n4.7 Other results \n... \n5.0 RECOMMENDATIONS\n...\n5.1 Recommend that a HMD for the B-1B not be pursued further as mechanized herein."
2,"A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf"" target=""_blank"">A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf"" target=""_blank"">A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf"" target=""_blank"">A Human Factors Study of a Helmet Mounted Display for the Pilots of the B-1B.pdf",30,"> The HMD provides a steering arrow directly to the DMPIs.\n> Having LAR depicted on ground puts your head down – looking through/below VSD\n> Adds SA to current displays.\n> Nice steering cues to the LAR.\n> Difficult to tell where the LAR / weapon footprints are – have to rely on arrow symbol to turn jet.\n> Look down angle unacceptable – though/below VSD.\n> The ability to simply look to the right or left and see the DMPIs was very useful. It will help pilots operationally because they can quickly see a baring and distance relationship between the LARs and DMPIs.\n> A little more difficult to fly and see the DMPIs.\n> When maneuvering, for threats it is important to know DMPI order (i.e. blocked weapons). A number needs to be added to diamond. More fusion required – distance to LAR etc. requires a more composite crosscheck.\n> Nice to have. Definitely needed, along with the LAR info.\n> Would add “G” display and possibly AOA, as it would be very easy to over G the jet with all of the heads up time and no G display.\n> Navigation info to the next steer point was good.\n> Either put the necessary navigation symbols on the HMD display or go with a head down display. Need more navigation symbology and numbers to actually navigate.\n> The airs